In [1]:
!mkdir data/indian_driving_coco
!mkdir data/indian_driving_coco/images
!mkdir data/indian_driving_coco/images/train
!mkdir data/indian_driving_coco/images/val
!mkdir data/indian_driving_coco/labels
!mkdir data/indian_driving_coco/labels/train
!mkdir data/indian_driving_coco/labels/val

In [2]:
import os
import json
from glob import glob

from tqdm import tqdm
from bs4 import BeautifulSoup

In [3]:
train_image_paths = glob('./data/indian_driving_dataset_train/images/train/*.jpg')
train_image_paths = sorted(train_image_paths)
len(train_image_paths)

9451

In [4]:
for train_image_path in tqdm(train_image_paths):
    from_train_image_path = train_image_path
    to_train_image_path = train_image_path.replace('indian_driving_dataset_train', 'indian_driving_coco')
    os.system(f'cp {train_image_path} {to_train_image_path}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9451/9451 [00:27<00:00, 342.62it/s]


In [5]:
val_image_paths = glob('./data/indian_driving_dataset_train/images/val/*.jpg')
val_image_paths = sorted(val_image_paths)
len(val_image_paths)

5392

In [6]:
for val_image_path in tqdm(val_image_paths):
    from_val_image_path = val_image_path
    to_val_image_path = val_image_path.replace('indian_driving_dataset_train', 'indian_driving_coco')
    os.system(f'cp {from_val_image_path} {to_val_image_path}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5392/5392 [00:17<00:00, 306.11it/s]


In [7]:
with open('./data/indian_driving_dataset_train/class_mapping.json', 'r') as file:
    class_map_dict = json.load(file)

class_map_dict

{'car': 0,
 'bus': 1,
 'autorickshaw': 2,
 'vehicle fallback': 3,
 'truck': 4,
 'motorcycle': 5,
 'rider': 6,
 'person': 7,
 'bicycle': 8,
 'animal': 9,
 'traffic sign': 10,
 'train': 11,
 'trailer': 12,
 'traffic light': 13,
 'caravan': 14}

In [8]:
def convert_to_coco_label_format(x1, y1, x2, y2, img_w, img_h):
    # Format: relative
    # object_index - x_center - y_center - width - height
    x = (x1 + x2) / 2 / img_w
    y = (y1 + y2) / 2 / img_h
    w = (x2 - x1) / img_w
    h = (y2 - y1) / img_h
    return x, y, w, h

In [9]:
def get_img_w_h(bs_data):
    w = int(bs_data.find('size').find('width').text)
    h = int(bs_data.find('size').find('height').text)
    return w, h

In [10]:
def get_img_label(obj_data):
    class_ = obj_data.find('name').text
    bbox = obj_data.find('bndbox')
    x1 = int(bbox.find('xmin').text)
    y1 = int(bbox.find('ymin').text)
    x2 = int(bbox.find('xmax').text)
    y2 = int(bbox.find('ymax').text)
    return class_, x1, y1, x2, y2

In [11]:
def prepare_coco_label(image_paths):
    for image_path in tqdm(image_paths):
        label_path = image_path.replace('images', 'labels').replace('.jpg', '.txt')
    
        with open(label_path, 'r') as f:
            bs_data = BeautifulSoup(f.read(), 'xml')
    
        img_w, img_h = get_img_w_h(bs_data)
        img_label_list = []
        for obj_data in bs_data.find_all('object'):
            class_, x1, y1, x2, y2 = get_img_label(obj_data)
            x, y, w, h = convert_to_coco_label_format(x1, y1, x2, y2, img_w, img_h)
            img_label_list.append(f'{class_map_dict[class_]} {x} {y} {w} {h}')
    
        coco_label_path = label_path.replace('indian_driving_dataset_train', 'indian_driving_coco')
        with open(coco_label_path, 'w') as file:
            file.write('\n'.join(img_label_list))

In [12]:
prepare_coco_label(train_image_paths)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9451/9451 [00:10<00:00, 940.78it/s]


In [13]:
prepare_coco_label(val_image_paths)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5392/5392 [00:05<00:00, 1004.02it/s]


In [14]:
def create_txt_path_file(root_path, dataset_name):
    file_path_list = glob(os.path.join(root_path, dataset_name, '*'))
    file_path_list = [path.replace('data/indian_driving_coco', '.') + '\n' for path in file_path_list]
    with open(f'data/indian_driving_coco/{dataset_name}.txt', 'w') as file:
        file.writelines(file_path_list)

In [15]:
create_txt_path_file('data/indian_driving_coco/images/', 'train')

In [16]:
create_txt_path_file('data/indian_driving_coco/images/', 'val')